In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.io import loadmat
from math import sqrt

In [2]:
def initializeWeights(n_in, n_out):
    epsilon = sqrt(6) / sqrt(n_in + n_out + 1)
    W = (np.random.rand(n_out, n_in + 1) * 2 * epsilon) - epsilon
    return W

In [3]:
def sigmoid(z):
    return (1.0 / (1.0 + np.exp(-z)))

In [4]:
def preprocess():
    pass

In [5]:
def nnObjFunction(params, *args):
    pass

In [6]:
def nnPredict(w1, w2, data):
    pass